提取同一簇的所有paper以及links构造network

In [1]:
import json

paperToCluster = {}
paperTotopic = {}
papers = []
paperNodes = []


with open("../output_data/jsonCluster_All_Coop10.json", "r", encoding="utf-8") as f:


    jsonData = json.load(f)  


with open("../output_data/papersToTopicInfo.json", "r", encoding="utf-8") as f:
    jsonDataTopic = json.load(f)

paperNodes = jsonDataTopic
print(len(paperNodes))

for node in jsonDataTopic:
    paperToCluster[node["id"]]=node["cluster"]
    paperTotopic[node["id"]] = node["topic"]
    papers.append(node["id"])
print(len(paperTotopic))
print(len(paperToCluster))
print(len(papers))
papers = list(set(papers))
print(len(papers))


2756
2756
2756
2756
2756


In [2]:
paperLinks=[]
paperEdges={}
for link in jsonData["links"]:
    s=link["source"]
    t=link["target"]
    if s in papers and t in papers and paperToCluster[s]==paperToCluster[t]:
        paperLinks.append(link)
        if paperTotopic[s]==paperTotopic[t]:
            if paperToCluster[s] not in paperEdges:
                paperEdges[paperToCluster[s]]={}
            if paperTotopic[s] not in paperEdges[paperToCluster[s]]:
                paperEdges[paperToCluster[s]][paperTotopic[s]]=[]
            paperEdges[paperToCluster[s]][paperTotopic[s]].append((s,t))
            # break
            # 节点 节点 簇号 主题号
# print(paperEdges)
# print(len(paperLinks))
print(len(paperEdges))
print(paperEdges[1])

10
{3: [('Global Topology of 3D Symmetric Tensor Fields', 'Feature Surfaces in Symmetric Tensor Fields Based on Eigenvalue Manifold'), ('Global Topology of 3D Symmetric Tensor Fields', 'Robust and Fast Extraction of 3D Symmetric Tensor Field Topology'), ('Robust and Fast Extraction of 3D Symmetric Tensor Field Topology', 'Feature Surfaces in Symmetric Tensor Fields Based on Eigenvalue Manifold'), ('Feature Curves and Surfaces of 3D Asymmetric Tensor Fields', 'Feature Surfaces in Symmetric Tensor Fields Based on Eigenvalue Manifold'), ('Feature Curves and Surfaces of 3D Asymmetric Tensor Fields', 'Robust and Fast Extraction of 3D Symmetric Tensor Field Topology'), ('DT-MRI Streamsurfaces Revisited', 'Feature Surfaces in Symmetric Tensor Fields Based on Eigenvalue Manifold')], 1: [('A Simple Pipeline for Coherent Grid Maps', 'Small Multiples with Gaps'), ('The Effectiveness of Interactive Visualization Techniques for Time Navigation of Dynamic Graphs on Large Displays', 'Event-Based Dyna

用最小生成树算法构建最小连通子图

In [3]:
import networkx as nx
minEdges=[]
EdgeInquire={}  # 查询当前边
# 创建无向图（所有边的权重相同）
for cluster in paperEdges:
    for topic in paperEdges[cluster]:
        G = nx.Graph()
        G.add_edges_from(paperEdges[cluster][topic])  # 添加边，每条边的权重默认是 1
        # 计算最小生成树
        MST = nx.minimum_spanning_tree(G)
        for edge in MST.edges():
            e={"source":edge[0],"target":edge[1],"cluster":cluster,"topic":topic}
            minEdges.append(e)
            if (cluster,topic) not in EdgeInquire:
                EdgeInquire[(cluster,topic)]=[]
            EdgeInquire[(cluster, topic)].append((edge[0], edge[1]))
# 输出最小生成树的边
# print("最小生成树的边:", MST.edges())

In [4]:
# 修改links内容 建立同簇的引用网络
for i in range(len(paperLinks)):
   s=paperLinks[i]["source"]
   t=paperLinks[i]["target"]
   paperLinks[i]["cluster"]=paperToCluster[s]
   paperLinks[i]["value"]=1
   if paperTotopic[s]==paperTotopic[t]:
      for k,v in EdgeInquire[(paperToCluster[s],paperTotopic[s])]:
         if k==s and v==t or k==t and v==s:
            paperLinks[i]["value"]=10
            paperLinks[i]["topic"]=paperTotopic[s]

In [5]:
jsonNetwork={}
jsonNetwork["nodes"]=paperNodes
jsonNetwork["links"]=paperLinks
jsonNetwork["edges"]=minEdges

with open("../output_data/papersCoop10.json", "w", encoding="utf-8") as f:
    json.dump(jsonNetwork, f, indent=4, ensure_ascii=False)